<a href="https://colab.research.google.com/github/nabilaaryanti/Twitter-data-processing/blob/main/Crawling_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Moduls

In [ ]:
!pip install unidecode twython tweepy beautifulsoup4 tika

     |████████████████████████████████| 235 kB 10.4 MB/s 
  Created wheel for tika: filename=tika-1.24-py3-none-any.whl size=32891 sha256=19b245915d4eac6ac9dfec9a2dd0cbf7b16a886fb726c4d6f59329563cac3feb
  Stored in directory: /root/.cache/pip/wheels/ec/2b/38/58ff05467a742e32f67f5d0de048fa046e764e2fbb25ac93f3
Successfully built tika


In [ ]:
import warnings; warnings.simplefilter('ignore')
import tweepy, json, nltk, urllib.request, requests
from urllib.request import Request, urlopen
from twython import TwythonStreamer
from bs4 import BeautifulSoup as bs
import pandas as pd
import os

## Connect to Twitter API

In [ ]:
Ck = '-' # consumer_key
Cs = '-' # consumer_secret
At = '-' # access_token
As = '-' # access_secret

In [ ]:
def twitter_connect(Ck, Cs, At, As, verbose = 0):
    try:
        auth = tweepy.OAuthHandler(Ck, Cs)
        auth.set_access_token(At, As)
        twitter = tweepy.API(auth, timeout=120)
        if verbose != 0:
            user = twitter.verify_credentials()
            print('Welcome "%s" you are now connected to twitter server' %user.name)
        return twitter
    except:
        print("Connection failed, please check your API keys or connection")
        return None

In [ ]:
# Koneksi ke twitter
twitter = twitter_connect(Ck, Cs, At, As, verbose = 1)

Welcome "ryanti" you are now connected to twitter server


## Crawling Data
Data akan diambil minimal sebanyak 30.000 tweet dengan kata kunci "Anthony Sinisuka Ginting", "Ginting", "ThomasCup2021" dan "Badminton" yang trending pada hari Jum'at, 15 Oktober 2021.

In [ ]:
# Max 100 tweet per "API call"
topic = 'Badminton'
N = 100 # jumlah tweet yang ingin diambil
bahasa = 'id'
T = twitter.search(q=topic, lang=bahasa, count=N, tweet_mode = 'extended')
data3 = [t._json for t in T]
print(' Berhasil mendapatkan {} tweets'.format(len(data3)))

 Berhasil mendapatkan 100 tweets


In [ ]:
# Next data
last_id = T[-1]._json['id'] - 1
last_id
T = twitter.search(q=topic, lang=bahasa, count=N, tweet_mode = 'extended', max_id=last_id)
data3.extend([t._json for t in T])

print('Jumlah data sekarang = ', len(data3))

## Cek Data

In [ ]:
# Datanya berbentuk JSON
tweets[0]['created_at'], tweets[0]['full_text']

('Fri Oct 15 13:02:16 +0000 2021',
 'RT @BadmintonTalk: Quarter Finals\nINA 1-0 MAS\n\nMS1: Anthony Sinisuka Ginting vs Lee Zii Jia\n\n21-15 21-17 \n \nYESSSSSSSSSSSSS MENANGGGGGGGGGG…')

In [ ]:
tweets[17000]

{'contributors': None,
 'coordinates': None,
 'created_at': 'Fri Oct 15 08:30:49 +0000 2021',
 'display_text_range': [12, 114],
 'entities': {'hashtags': [],
  'symbols': [],
  'urls': [],
  'user_mentions': [{'id': 132276059,
    'id_str': '132276059',
    'indices': [0, 11],
    'name': 'A. Ainur Rohman',
    'screen_name': 'ainurohman'}]},
 'favorite_count': 10,
 'favorited': False,
 'full_text': '@ainurohman Ginting dan minions yuk main bar bar di awal, biar jojo, Fajri sama mas vito ketularan bar barnya juga',
 'geo': None,
 'id': 1448929342503800834,
 'id_str': '1448929342503800834',
 'in_reply_to_screen_name': 'ainurohman',
 'in_reply_to_status_id': 1448922737028648961,
 'in_reply_to_status_id_str': '1448922737028648961',
 'in_reply_to_user_id': 132276059,
 'in_reply_to_user_id_str': '132276059',
 'is_quote_status': False,
 'lang': 'in',
 'metadata': {'iso_language_code': 'in', 'result_type': 'recent'},
 'place': None,
 'retweet_count': 0,
 'retweeted': False,
 'source': '<a hre

In [ ]:
print('tweet pertama oleh "{}" : "{}"'.format(tweets[3]['user']['screen_name'],tweets[0]['full_text']))

tweet pertama oleh "athorumbo" : "Jadi, Siaga Jiwa Raga menjadi sangat penting di era pandemi Corona Virus Desease (Covid-19) sekarang ini
#SantriSiagaJiwaRaga
Logo Hari Santri 2021 https://t.co/PUDpHiawnH"


In [ ]:
def saveTweets(data3, file='Tweets_bad.json'): #in Json Format
    with open(file, 'w') as f:
        for t in data3:
            try:
                f.write(json.dumps(t)+'\n')
            except:
                pass
              
# Menyimpan hasil crawling twitter
fileName = '/content/drive/MyDrive/Colab Notebooks/badminton.json'
saveTweets(data3,file=fileName)
print('Saved to '+fileName)

Saved to /content/drive/MyDrive/Colab Notebooks/badminton.json


In [ ]:
def loadTweets(file='Tweets_bad.json'):
    f=open(file,encoding='utf-8', errors ='ignore', mode='r')
    T=f.readlines();f.close()
    for i,t in enumerate(T):
        T[i] = json.loads(t.strip())
    return T

# Me-load kembali jika (misal) analisa ingin dilakukan di lain waktu
# Sengaja nama variabelnya saya bedakan (T2)
T2 = loadTweets(file='/content/drive/MyDrive/Colab Notebooks/badminton.json')
print('tweet pertama oleh "{}" : "{}"'.format(T2[0]['user']['screen_name'],T2[0]['full_text']))

tweet pertama oleh "melourip" : "Penat aku tengok badminton"


In [ ]:
T2[8]

{'contributors': None,
 'coordinates': None,
 'created_at': 'Fri Oct 15 13:33:54 +0000 2021',
 'display_text_range': [0, 140],
 'entities': {'hashtags': [],
  'symbols': [],
  'urls': [],
  'user_mentions': [{'id': 1013242098697494528,
    'id_str': '1013242098697494528',
    'indices': [3, 15],
    'name': 'farisha',
    'screen_name': 'FarishaChaa'}]},
 'favorite_count': 0,
 'favorited': False,
 'full_text': 'RT @FarishaChaa: Member sedap makan dekat foodcourt sambil layan orang main badminton ttbe surat sampai dari gf orang....🙃 https://t.co/LJv…',
 'geo': None,
 'id': 1449005616421240835,
 'id_str': '1449005616421240835',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'in',
 'metadata': {'iso_language_code': 'in', 'result_type': 'recent'},
 'place': None,
 'retweet_count': 230,
 'retweeted': False,
 'retweeted_status': {'contribu